In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import sympy as sp

In [31]:
xs_dot, A_p, A_s, xp_dot = sp.symbols(r'\dot{x}_s A_p A_s \dot{x_p}')
m_r, A_v, p_r, gamma, R, T = sp.symbols('m_r A_v p_r gamma R T')
m_p, p_p, V_p = sp.symbols('m_p p_p V_p')
mp_dot, mr_dot = sp.symbols(r'\dot{m}_p \dot{m}_r')
p_r, m_r, gamma, R, T, V_r = sp.symbols('p_r m_r gamma R T V_r')
rho_p, V_p, gamma, c_p, m_p, T, A_p, x_p, A_s, p_p, p_s, m_w, m_piston, g, m_water, x_s = sp.symbols(
'rho_p V_p gamma c_p m_p T A_p x_p A_s p_p p_s m_w m_piston g m_water x_s')


eq1 = sp.Eq(xs_dot,A_p/A_s * xp_dot)
eq2 = sp.Eq(
    sp.Derivative(m_r, 't'),
    -A_v * p_r * sp.sqrt(gamma / (R * T)) * ((2 / (gamma + 1)) ** ((gamma + 1) / (2 * (gamma - 1))))
)

eq3 = 3
eq4 = sp.Eq(mp_dot, -mr_dot)
eq5 = sp.Eq(p_r,mr_dot* (gamma * R * T) / V_r)

# Define the equations
eq6 = sp.Eq(sp.Derivative(p_p, 't'), (1/V_p) * ((gamma - 1) * c_p * m_p * T - gamma * p_p * A_p * sp.Derivative(x_p, 't')))
eq7 = sp.Eq(sp.Derivative(x_p, ('t', 2)), (A_p * p_p - A_s * p_s - (m_w + m_piston) * g) / m_piston)
eq8 = sp.Eq(sp.Derivative(x_s, ('t', 2)), (-A_s * p_s + A_p * p_p - m_w * g) / m_water)

# Display the equations
display(eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8)

Eq(\dot{x}_s, A_p*\dot{x_p}/A_s)

Eq(Derivative(m_r, t), -A_v*p_r*(2/(gamma + 1))**((gamma + 1)/(2*gamma - 2))*sqrt(gamma/(R*T)))

3

Eq(\dot{m}_p, -\dot{m}_r)

Eq(p_r, R*T*\dot{m}_r*gamma/V_r)

Eq(Derivative(p_p, t), (-A_p*gamma*p_p*Derivative(x_p, t) + T*c_p*m_p*(gamma - 1))/V_p)

Eq(Derivative(x_p, (t, 2)), (A_p*p_p - A_s*p_s - g*(m_piston + m_w))/m_piston)

Eq(Derivative(x_s, (t, 2)), (A_p*p_p - A_s*p_s - g*m_w)/m_water)